# 要約 
このJupyter Notebookでは、Metaの**Llama3**（8Bモデル）をファインチューニングする方法を示しています。具体的には、**QLoRA**や**trl**ライブラリの**SFTTrainer**を使用して、大規模言語モデル（LLM）のパラメータ効率の高い調整（PEFT）を行うプロセスに取り組んでいます。

### 問題の対象
ノートブックは主に、Chatbot Arenaコンペティションに関連するデータセットを用いて、どのモデルの応答がユーザーに好まれるかを予測する問題に取り組んでいます。具体的には、2つの異なるモデル（応答Aと応答B）からの出力に基づき、どちらが好まれるかを分類するタスクです。

### 使用している手法
- **QLoRA**: Low-Rank Adaptation（LoRA）に基づき、ファインチューニング中のメモリ使用量を削減しつつ、モデルのパフォーマンスを維持または向上させる手法です。これにより、大規模モデルを少ないGPUで効率的に訓練できます。
- **SFTTrainer**: Hugging Faceの**trl**ライブラリによって提供されるシンプルなAPIを用いており、ファインチューニングプロセスを容易にします。
- **PEFT**: 事前訓練済みモデルのパラメータを再利用することで、計算資源と時間を節約しながら、タスク特有の性能を引き上げる技術です。

### 利用ライブラリ
- **Transformers**: 自然言語処理のための高度なモデルを提供し、モデルのロードやファインチューニングが可能です。
- **Torch**: ディープラーニングフレームワークとして使用され、モデルの構築や訓練に利用されています。
- **PEFT**: パラメータ効率の良いファインチューニングを行うためのライブラリです。

このノートブックは、事前データ処理、モデルの設定、学習のトレーニングを行い、最後にはモデルのパフォーマンスを評価することで完了します。最終的に、予測結果をサンプルとして表示することも含まれており、コンペティションの要件に従ったデータサイエンスプロジェクトの構築に役立つ内容となっています。

---


# 用語概説 
以下は、初心者がつまずきそうな専門用語に関する簡単な解説です。特に、実務経験のない方には馴染みが薄く、ノートブック特有のドメイン知識が関連する用語に焦点を当てています。

1. **QLoRA（Quantized LoRA）**:
   QLoRAは、LoRA（Low-Rank Adaptation）のアプローチに基づき、モデルの性能を維持または向上させつつ、メモリ使用量を効率的に削減する手法です。量子化を組み合わせることで、少ないGPUリソースでも大規模モデルをファインチューニングできるようになります。特にQA（質問応答）タスクでの応用が期待されています。

2. **PEFT（Parameter-Efficient Fine-Tuning）**:
   PEFTは、特定の下流タスクについて事前訓練したモデルの性能を向上させるための方法です。これを実現するために、全体のモデルをゼロから訓練するのではなく、小規模なデータセットを使用して特定の部分だけを調整します。計算資源の節約やトレーニング時間の短縮が可能です。

3. **SFTTrainer**:
   SFTTrainerは、Social Fine-Tuning（SFT）のためのTrainerクラスです。Hugging Faceのtrlライブラリに含まれ、モデルをファインチューニングするための簡単かつ効率的なAPIを提供します。このクラスを利用することで、複雑な設定を簡略化し、ファインチューニングを容易に行えます。

4. **UltraChat200k**:
   UltraChat200kは、大規模な対話データセットであり、約140万の対話を含みます。自然言語理解や生成、対話システムの研究にとって貴重なリソースです。異なるフォーマットが提供され、実験やモデル評価に利用があります。

5. **ロギング（Logging）**:
   機械学習におけるロギングは、訓練過程や評価結果を記録することを指します。トレーニングのパラメータや状態を追跡し、後の解析やモデルのチューニングに役立てます。

6. **二重量子化（Double Quantization）**:
   量子化は、モデルのサイズを圧縮する手法ですが、二重量子化は複数の段階で量子化を行うことでさらなる圧縮を実現します。この結果、メモリ使用量が削減され、計算効率が向上します。

7. **ドロップアウト（Dropout）**:
   ドロップアウトは、過学習を防ぐためのテクニックで、トレーニング中に一定の確率でニューロンを無効化します。これにより、モデルは特定のパターンに依存することが難しくなり、汎化能力が向上します。

8. **ガーベジコレクション（Garbage Collection）**:
   プログラミングにおいて、使用しなくなったメモリを自動的に解放する仕組みを指します。特に、Pythonなどの言語で無駄なメモリを掃除し、リソースを効率的に管理します。

9. **クラス重み（Class Weights）**:
   クラス重みは、モデルの損失関数において特定のクラスの重要度を調整するために使用されます。データセットが不均衡な場合に、少数派クラスの影響を強化するためによく用いられます。

これらの専門用語に精通することは、ノートブックや深層学習モデルの理解を深める上で重要です。特に、BERTやGPTなどの大規模言語モデルに関連する技術とその効率的な活用方法は、今日のAI開発において非常に価値があります。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Introduction


In this notebook we demostrate how to finetune **Llama3** (8B) model from Meta using **QLoRA** and **SFTTrainer** from **tlr**.

## What is Llama3?

Llama3 is the latest release of open-source LLMs from Meta, with features pretrained and instruction-fine-tuned language models with 8B and 70B parameters.

## What is LoRA?

LoRA stands for Low-Rank Adaptation. It is a method used to fine-tune large language models (LLMs) by freezing the weights of the LLM and injecting trainable rank-decomposition matrices. The number of trainable parameters during fine-tunning will decrease therefore considerably. According to LoRA paper, this number decreases 10,000 times, and the computational resources size decreases 3 times.


## What is QLoRA?

QLoRA builds on LoRA by incorporating quantization techniques to further reduce memory usage while maintaining, or even enhancing, model performance. With QLoRA it is possible to finetune a 70B parameter model that requires 36 GPUs with only 2!


## What is PEFT?

Parameter-efficient Fine-tuning (PEFT) is a technique used in Natural Language Processing (NLP) to improve the performance of pre-trained language models on specific downstream tasks. It involves reusing the pre-trained model’s parameters and fine-tuning them on a smaller dataset, which saves computational resources and time compared to training the entire model from scratch. PEFT achieves this efficiency by freezing some of the layers of the pre-trained model and only fine-tuning the last few layers that are specific to the downstream task.

## What is SFTTrainer?

SFT in SFTTrainer stands for supervised fine-tuning. The **trl** (Transformer Reinforcement Learning) library from HuggingFace provides a simple API to fine-tune models using SFTTrainer.

## What is UltraChat200k?  

UltraChat-200k is an invaluable resource for natural language understanding, generation and dialog system research. With 1.4 million dialogues spanning a variety of topics, this parquet-formatted dataset offers researchers four distinct formats to aid in their studies: test_sft, train_sft, train_gen and test_gen. More details [here](https://www.kaggle.com/datasets/thedevastator/ultrachat-200k-nlp-dataset).

## Inspiration

For this notebook, I took inspiration from several sources:
* [Efficiently fine-tune Llama 3 with PyTorch FSDP and Q-Lora](https://www.philschmid.de/fsdp-qlora-llama3)  
* [Fine-tuning LLMs using LoRA](https://medium.com/@rajatsharma_33357/fine-tuning-llama-using-lora-fb3f48a557d5)  
* [Fine-tuning Llama-3–8B-Instruct QLORA using low cost resources](https://medium.com/@avishekpaul31/fine-tuning-llama-3-8b-instruct-qlora-using-low-cost-resources-89075e0dfa04)  
* [Llama2 Fine-Tuning with Low-Rank Adaptations (LoRA) on Gaudi 2 Processors](https://eduand-alvarez.medium.com/llama2-fine-tuning-with-low-rank-adaptations-lora-on-gaudi-2-processors-52cf1ee6ce11)  

</div>
<div class="column-right">

# 日本語訳

# はじめに

このノートブックでは、**QLoRA** と **tlr** の **SFTTrainer** を使用して、Meta の **Llama3** (8B) モデルをファインチューニングする方法を示します。

## Llama3 とは？

Llama3 は、Meta が提供するオープンソースの最新の大規模言語モデル（LLM）で、8B および 70B パラメータを持つ事前訓練済みおよび指示に基づいてファインチューニングされた言語モデルを特徴としています。

## LoRA とは？

LoRA は Low-Rank Adaptation の略です。これは、大規模言語モデル（LLMs）をファインチューニングするための方法で、LLM の重みを固定し、トレーニング可能なランク分解行列を注入します。ファインチューニング中のトレーニング可能なパラメータの数は、かなり減少します。LoRA の論文によると、この数は 10,000 倍減少し、計算資源のサイズは 3 倍減少するとされています。

## QLoRA とは？

QLoRA は LoRA に基づいて、モデルの性能を維持または向上させながらメモリ使用量をさらに削減するために量子化手法を組み合わせたものです。QLoRA を使用すると、36 台の GPU が必要な 70B パラメータモデルをわずか 2 台の GPU でファインチューニングすることが可能になります！

## PEFT とは？

パラメータ効率の良いファインチューニング（PEFT）は、特定の下流タスクにおいて事前訓練した言語モデルの性能を向上させるための技術です。これは、事前訓練済みモデルのパラメータを再利用し、より小さなデータセットでファインチューニングすることで、全モデルをゼロからトレーニングするのに比べて、計算資源と時間を節約します。PEFT は、事前訓練済みモデルのいくつかの層を固定し、下流タスクに特有の最後の数層だけをファインチューニングすることによって、この効率が実現されます。

## SFTTrainer とは？

SFTTrainer の SFT はフォローアップのファインチューニングを指します。HuggingFace の **trl** (Transformer Reinforcement Learning) ライブラリは、SFTTrainer を使用してモデルをファインチューニングするためのシンプルな API を提供します。

## UltraChat200k とは？

UltraChat-200k は、自然言語理解、生成、および対話システム研究にとって非常に貴重なリソースです。多数のトピックにわたる 1.4 百万の対話を含み、この parquet 形式のデータセットは、研究者が研究を支援するために 4 つの異なるフォーマットを提供します: test_sft, train_sft, train_gen, test_gen。詳細は [こちら](https://www.kaggle.com/datasets/thedevastator/ultrachat-200k-nlp-dataset)をご覧ください。

## インスピレーション

このノートブックでは、いくつかの情報源からインスピレーションを得ました：
* [PyTorch FSDP と Q-Lora を使用して Llama 3 を効率的にファインチューニングする](https://www.philschmid.de/fsdp-qlora-llama3)  
* [LoRA を使用した LLM のファインチューニング](https://medium.com/@rajatsharma_33357/fine-tuning-llama-using-lora-fb3f48a557d5)  
* [低コストリソースを使用した Llama-3–8B–Instruct QLORA のファインチューニング](https://medium.com/@avishekpaul31/fine-tuning-llama-3-8b-instruct-qlora-using-low-cost-resources-89075e0dfa04)  
* [Gaudi 2 プロセッサ上での Low-Rank Adaptations (LoRA) を用いた Llama2 のファインチューニング](https://eduand-alvarez.medium.com/llama2-fine-tuning-with-low-rank-adaptations-lora-on-gaudi-2-processors-52cf1ee6ce11)  

# ライブラリのインストールとインポート

</div>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Install and import libraries

</div>
<div class="column-right">

# 日本語訳

!pip install -q -U bitsandbytes  # bitsandbytesライブラリをインストールします。これは、低ランクモデルのファインチューニングに役立つツールです。
!pip install -q -U transformers  # transformersライブラリをインストールします。このライブラリは、自然言語処理のための高度な機械学習モデルを提供します。
!pip install -q -U peft  # パラメータ効率の良いファインチューニングを行うためのpeftライブラリをインストールします。
!pip install -q -U accelerate  # accelerateライブラリをインストールします。これは、訓練の高速化や効率的な実行を支援します。
!pip install -q -U datasets  # データセットライブラリをインストールします。さまざまなデータセットにアクセスして利用するためのライブラリです。
!pip install -q -U trl  # トランスフォーマーの強化学習ライブラリであるtrlをインストールします。モデルの強化学習に役立ちます。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
```

</div>
<div class="column-right">

# 日本語訳

```python
import pandas as pd  # データ操作を容易にするためのpandasライブラリをインポートします。これはデータフレームの作成や操作に役立ちます。
from torch.utils.data import Dataset  # PyTorchのデータセットクラスをインポートします。カスタムデータセットを作成するために使用します。
from torch.utils.data import DataLoader  # PyTorchのデータローダークラスをインポートします。バッチ処理やデータのシャッフルをサポートします。
```

</div>
</details>

In [ ]:
import pandas as pd  # データ操作を容易にするためのpandasライブラリをインポートします。これはデータフレームの作成や操作に役立ちます。
from torch.utils.data import Dataset  # PyTorchのデータセットクラスをインポートします。カスタムデータセットを作成するために使用します。
from torch.utils.data import DataLoader  # PyTorchのデータローダークラスをインポートします。バッチ処理やデータのシャッフルをサポートします。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
```

</div>
<div class="column-right">

# 日本語訳

```python
# from kaggle_secrets import UserSecretsClient  # Kaggleの秘密情報にアクセスするためのUserSecretsClientをインポートします。
# user_secrets = UserSecretsClient()  # UserSecretsClientのインスタンスを作成します。
# wandb_key = user_secrets.get_secret("wandb_api")  # WandB（Weights & Biases）APIキーを取得します。このキーはWandBの機能を利用するために必要です。
# import wandb  # WandBライブラリをインポートします。これは、実験の追跡や可視化に役立ちます。
# ! wandb login $wandb_key  # 取得したWandB APIキーを使ってWandBにログインします。これにより、実験データをWandBにアップロードできるようになります。
```

</div>
</details>

In [ ]:
# from kaggle_secrets import UserSecretsClient  # Kaggleの秘密情報にアクセスするためのUserSecretsClientをインポートします。
# user_secrets = UserSecretsClient()  # UserSecretsClientのインスタンスを作成します。
# wandb_key = user_secrets.get_secret("wandb_api")  # WandB（Weights & Biases）APIキーを取得します。このキーはWandBの機能を利用するために必要です。
# import wandb  # WandBライブラリをインポートします。これは、実験の追跡や可視化に役立ちます。
# ! wandb login $wandb_key  # 取得したWandB APIキーを使ってWandBにログインします。これにより、実験データをWandBにアップロードできるようになります。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# wandb_key = user_secrets.get_secret("wandb_api")
# import wandb
# ! wandb login $wandb_key
```

</div>
<div class="column-right">

# 日本語訳

```python
import os  # オペレーティングシステムの機能を提供するosライブラリをインポートします。ファイルパスなどの操作に使用します。
import torch  # PyTorchライブラリをインポートします。ディープラーニングモデルの構築とトレーニングに使用します。
from time import time  # 現在の時刻を取得するためのtime関数をインポートします。トレーニング時間の計測に使用します。
from datasets import load_dataset  # datasetsライブラリからload_dataset関数をインポートします。データセットを簡単に読み込むために使用します。
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training  # PEFT（パラメータ効率の良いファインチューニング）に関連するクラスや関数をインポートします。
from transformers import (  # transformersライブラリから様々なクラスや関数をインポートします。
    AutoConfig,  # モデルの自動設定を行うためのクラスをインポートします。
    AutoModelForCausalLM,  # 自然言語生成用の因果モデルを自動で取得するためのクラスをインポートします。
    AutoTokenizer,  # モデルに適したトークナイザーを自動で取得するためのクラスをインポートします。
    BitsAndBytesConfig,  # メモリ効率化のためのビットおよびバイト設定を管理するクラスをインポートします。
    AutoTokenizer,  # トークナイザーを再度インポート（重複）していますが、必要に応じて実際のインポートを行います。
    TrainingArguments,  # トレーニングの際の引数を設定するためのクラスをインポートします。
    AutoModelForSequenceClassification,  # シーケンス分類用のモデルを自動で取得するためのクラスをインポートします。
    Trainer  # モデルのトレーニングを管理するための教師クラスをインポートします。
)
from trl import SFTTrainer, setup_chat_format  # SFTTrainerクラスとチャット形式を設定する関数をインポートします。ファインチューニングに使用します。
import numpy as np  # 数値計算を効率的に行うためのnumpyライブラリをインポートします。配列や行列の操作に便利です。
```

</div>
</details>

In [ ]:
import os  # オペレーティングシステムの機能を提供するosライブラリをインポートします。ファイルパスなどの操作に使用します。
import torch  # PyTorchライブラリをインポートします。ディープラーニングモデルの構築とトレーニングに使用します。
from time import time  # 現在の時刻を取得するためのtime関数をインポートします。トレーニング時間の計測に使用します。
from datasets import load_dataset  # datasetsライブラリからload_dataset関数をインポートします。データセットを簡単に読み込むために使用します。
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training  # PEFT（パラメータ効率の良いファインチューニング）に関連するクラスや関数をインポートします。
from transformers import (  # transformersライブラリから様々なクラスや関数をインポートします。
    AutoConfig,  # モデルの自動設定を行うためのクラスをインポートします。
    AutoModelForCausalLM,  # 自然言語生成用の因果モデルを自動で取得するためのクラスをインポートします。
    AutoTokenizer,  # モデルに適したトークナイザーを自動で取得するためのクラスをインポートします。
    BitsAndBytesConfig,  # メモリ効率化のためのビットおよびバイト設定を管理するクラスをインポートします。
    AutoTokenizer,  # トークナイザーを再度インポート（重複）していますが、必要に応じて実際のインポートを行います。
    TrainingArguments,  # トレーニングの際の引数を設定するためのクラスをインポートします。
    AutoModelForSequenceClassification,  # シーケンス分類用のモデルを自動で取得するためのクラスをインポートします。
    Trainer  # モデルのトレーニングを管理するための教師クラスをインポートします。
)
from trl import SFTTrainer, setup_chat_format  # SFTTrainerクラスとチャット形式を設定する関数をインポートします。ファインチューニングに使用します。
import numpy as np  # 数値計算を効率的に行うためのnumpyライブラリをインポートします。配列や行列の操作に便利です。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import os
import torch
from time import time
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    AutoModelForSequenceClassification,
    Trainer
)
from trl import SFTTrainer,setup_chat_format
import numpy as np
```

</div>
<div class="column-right">

# 日本語訳

```python
class CFG:  # コンフィグレーションクラスを定義します。モデルのトレーニングに関連する設定を保持します。
    NUM_EPOCHS = 1  # エポック数を設定します。モデルの学習を何回繰り返すかを決定します。
    BATCH_SIZE = 16  # バッチサイズを設定します。一度に処理するサンプルの数です。
    DROPOUT = 0.05  # ドロップアウト率を設定します。過学習を防ぐために、一部のニューロンの出力を確率的に無効にします。
    MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'  # 使用するモデルのパスを定義します。

    SEED = 2024  # ランダムシードを設定します。再現性のある結果を得るために使用します。
    MAX_LENGTH = 1024  # 最大入力長を設定します。モデルへの入力として受け入れる最大トークン数です。
    NUM_WARMUP_STEPS = 128  # ウォームアップステップの数を設定します。学習率を徐々に増加させるステップ数です。
    LR_MAX = 5e-5  # 学習率の最大値を設定します。これにより、モデルが効果的に学習できるようになります。
    NUM_LABELS = 3  # 分類タスクにおけるラベルの数を設定します。
    LORA_RANK = 4  # LoRAのランクを設定します。モデルのランク近似の次元を決定します。
    LORA_ALPHA = 8  # LoRAで使用するスケーリング係数を設定します。
    LORA_MODULES = ['o_proj', 'v_proj']  # LoRAを適用するモデルのモジュール名をリストとして定義します。ここでは出力プロジェクションとバリュー仮想プロジェクションを指定しています。
```

</div>
</details>

In [ ]:
class CFG:  # コンフィグレーションクラスを定義します。モデルのトレーニングに関連する設定を保持します。
    NUM_EPOCHS = 1  # エポック数を設定します。モデルの学習を何回繰り返すかを決定します。
    BATCH_SIZE = 16  # バッチサイズを設定します。一度に処理するサンプルの数です。
    DROPOUT = 0.05  # ドロップアウト率を設定します。過学習を防ぐために、一部のニューロンの出力を確率的に無効にします。
    MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'  # 使用するモデルのパスを定義します。

    SEED = 2024  # ランダムシードを設定します。再現性のある結果を得るために使用します。
    MAX_LENGTH = 1024  # 最大入力長を設定します。モデルへの入力として受け入れる最大トークン数です。
    NUM_WARMUP_STEPS = 128  # ウォームアップステップの数を設定します。学習率を徐々に増加させるステップ数です。
    LR_MAX = 5e-5  # 学習率の最大値を設定します。これにより、モデルが効果的に学習できるようになります。
    NUM_LABELS = 3  # 分類タスクにおけるラベルの数を設定します。
    LORA_RANK = 4  # LoRAのランクを設定します。モデルのランク近似の次元を決定します。
    LORA_ALPHA = 8  # LoRAで使用するスケーリング係数を設定します。
    LORA_MODULES = ['o_proj', 'v_proj']  # LoRAを適用するモデルのモジュール名をリストとして定義します。ここでは出力プロジェクションとバリュー仮想プロジェクションを指定しています。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
class CFG:
    NUM_EPOCHS = 1
    BATCH_SIZE = 16
    DROPOUT = 0.05 
    MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
    
    SEED = 2024 
    MAX_LENGTH = 1024 
    NUM_WARMUP_STEPS = 128
    LR_MAX = 5e-5 
    NUM_LABELS = 3 
    LORA_RANK = 4
    LORA_ALPHA = 8
    LORA_MODULES = ['o_proj', 'v_proj']
```

</div>
<div class="column-right">

# 日本語訳

```python
# データセットの準備
```

</div>
</details>

# データセットの準備

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Prepare Dataset

</div>
<div class="column-right">

# 日本語訳

train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')  # トレーニングデータセットをCSVファイルから読み込みます。

def process(input_str):  # 入力文字列を処理する関数を定義します。
    stripped_str = input_str.strip('[]')  # 文字列の前後の角括弧を取り除きます。
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 文字列を分割し、余分な引用符を取り除きます。
    return  ' '.join(sentences)  # 分割した文を空白で結合して返します。

# 'prompt'、'response_a'、'response_b' 列に対して process 関数を適用します。
train.loc[:, 'prompt'] = train['prompt'].apply(process)  # プロンプト列の処理。
train.loc[:, 'response_a'] = train['response_a'].apply(process)  # モデルAの応答を処理。
train.loc[:, 'response_b'] = train['response_b'].apply(process)  # モデルBの応答を処理。

# トレーニング用に 'Null' をドロップします。
indexes = train[(train.response_a == 'null') & (train.response_b == 'null')].index  # 応答Aと応答Bが両方とも'null'である行のインデックスを取得します。
train.drop(indexes, inplace=True)  # その行をデータフレームから削除します。
train.reset_index(inplace=True, drop=True)  # インデックスをリセットします。
train.loc[:, 'label'] = np.argmax(train[['winner_model_a','winner_model_b','winner_tie']].values, axis=1)  # 各行に対して勝者モデルのラベルを計算して追加します。

print(f"Total {len(indexes)} Null response rows dropped")  # ドロップした行数を表示します。
print('Total train samples: ', len(train))  # 残りのトレーニングサンプル数を表示します。

# テキスト形式にデータを整形します。
train['text'] = 'User prompt: ' + train['prompt'] +  '\n\nModel A :\n' + train['response_a'] +'\n\n--------\n\nModel B:\n'  + train['response_b']
print(train['text'][4])  # フォーマットされたテキストの5番目のサンプルを表示します。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

train.loc[:, 'prompt'] = train['prompt'].apply(process)
train.loc[:, 'response_a'] = train['response_a'].apply(process)
train.loc[:, 'response_b'] = train['response_b'].apply(process)

# Drop 'Null' for training
indexes = train[(train.response_a == 'null') & (train.response_b == 'null')].index
train.drop(indexes, inplace=True)
train.reset_index(inplace=True, drop=True)
train.loc[:, 'label'] = np.argmax(train[['winner_model_a','winner_model_b','winner_tie']].values, axis=1)

print(f"Total {len(indexes)} Null response rows dropped")
print('Total train samples: ', len(train))

train['text'] = 'User prompt: ' + train['prompt'] +  '\n\nModel A :\n' + train['response_a'] +'\n\n--------\n\nModel B:\n'  + train['response_b']
print(train['text'][4])
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルの初期化

使用されるモデルは以下の通りです：

* **モデル**: Llama3  
* **フレームワーク**: Transformers  
* **サイズ**: 8B  
* **タイプ**: 8b-chat-hf (hfはHuggingFaceを指します)  
* **バージョン**: V1  
```

</div>
</details>

# モデルの初期化

使用されるモデルは以下の通りです：

* **モデル**: Llama3  
* **フレームワーク**: Transformers  
* **サイズ**: 8B  
* **タイプ**: 8b-chat-hf (hfはHuggingFaceを指します)  
* **バージョン**: V1  

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Initialize model


The model used is:

* **Model**: Llama3  
* **Framework**: Transformers   
* **Size**: 8B   
* **Type**: 8b-chat-hf (hf stands for HuggingFace). 
* **Version**: V1  

</div>
<div class="column-right">

# 日本語訳

from transformers import BitsAndBytesConfig, AutoModelForSequenceClassification  # ビットおよびバイトの設定とシーケンス分類用のモデルをインポートします。

# 量子化設定を定義します。
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4ビットでモデルをロードするように設定します。
    bnb_4bit_quant_type='nf4',  # 4ビット量子化のタイプを設定します。ここでは「nf4」を指定します。
    bnb_4bit_use_double_quant=True,  # 二重量子化を使用する設定です。これにより、さらにメモリを節約します。
    bnb_4bit_compute_dtype=torch.bfloat16  # 計算時に使用するデータ型をtorchのbfloat16に設定します。これにより、計算効率が向上します。
)

model_name = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"  # 使用するモデルの名前とパスを指定します。

# 特徴を持つモデルを事前訓練済みから読み込みます。
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,  # 読み込むモデルの名前。
    quantization_config=quantization_config,  # 量子化設定を適用します。
    num_labels=4,  # モデルで扱うラベルの数を設定します。
    device_map='auto'  # デバイスマップを自動で設定し、利用可能なGPUを使用します。
)

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from transformers import BitsAndBytesConfig, AutoModelForSequenceClassification

quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, 
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True, 
    bnb_4bit_compute_dtype = torch.bfloat16 
)

model_name = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=4,
    device_map='auto'
)
```

</div>
<div class="column-right">

# 日本語訳

```python
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model  # LoRA関連のクラスと関数をインポートします。

# LoRAの設定を定義します。
lora_config = LoraConfig(
    r=16,  # LoRAのランクを設定します。ここでは16を指定します。
    lora_alpha=8,  # LoRAで使用するスケーリング係数を設定します。
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'],  # LoRAを適用するターゲットモジュールを指定します。ここでは、クエリ、キー、バリュー、および出力プロジェクションを指定します。
    lora_dropout=0.05,  # LoRAのドロップアウト率を設定します。これにより、過学習を防ぎます。
    bias='none',  # バイアスの設定をしていません。バイアスを使用しないことを示します。
    task_type='SEQ_CLS'  # タスクのタイプをシーケンス分類に設定します。
)

# モデルをkbitトレーニング用に準備します。
model = prepare_model_for_kbit_training(model)  
# LoRA設定を使用してモデルを取得（適用）します。
model = get_peft_model(model, lora_config)
```

</div>
</details>

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model  # LoRA関連のクラスと関数をインポートします。

# LoRAの設定を定義します。
lora_config = LoraConfig(
    r=16,  # LoRAのランクを設定します。ここでは16を指定します。
    lora_alpha=8,  # LoRAで使用するスケーリング係数を設定します。
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'],  # LoRAを適用するターゲットモジュールを指定します。ここでは、クエリ、キー、バリュー、および出力プロジェクションを指定します。
    lora_dropout=0.05,  # LoRAのドロップアウト率を設定します。これにより、過学習を防ぎます。
    bias='none',  # バイアスの設定をしていません。バイアスを使用しないことを示します。
    task_type='SEQ_CLS'  # タスクのタイプをシーケンス分類に設定します。
)

# モデルをkbitトレーニング用に準備します。
model = prepare_model_for_kbit_training(model)  
# LoRA設定を使用してモデルを取得（適用）します。
model = get_peft_model(model, lora_config)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

lora_config = LoraConfig(
    r = 16, 
    lora_alpha = 8,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, 
    bias = 'none',
    task_type = 'SEQ_CLS'
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
```

</div>
<div class="column-right">

# 日本語訳

```python
from transformers import AutoTokenizer  # トークナイザーを自動で取得するためのクラスをインポートします。

# 事前訓練済みのトークナイザーをモデル名から読み込みます。このとき、トークンの前にスペースを追加する設定をします。
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

# パディングトークンIDを終了トークンIDに設定します。これにより、パディング時に同じトークンを使用します。
tokenizer.pad_token_id = tokenizer.eos_token_id
# パディングトークンを終了トークンに設定します。これにより、パディング時に使用するトークンを明示的に指定します。
tokenizer.pad_token = tokenizer.eos_token
```

</div>
</details>

In [ ]:
from transformers import AutoTokenizer  # トークナイザーを自動で取得するためのクラスをインポートします。

# 事前訓練済みのトークナイザーをモデル名から読み込みます。このとき、トークンの前にスペースを追加する設定をします。
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

# パディングトークンIDを終了トークンIDに設定します。これにより、パディング時に同じトークンを使用します。
tokenizer.pad_token_id = tokenizer.eos_token_id
# パディングトークンを終了トークンに設定します。これにより、パディング時に使用するトークンを明示的に指定します。
tokenizer.pad_token = tokenizer.eos_token

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token
```

</div>
<div class="column-right">

# 日本語訳

```python
model.config.pad_token_id = tokenizer.pad_token_id  # モデルの設定にパディングトークンIDを追加します。
model.config.use_cache = False  # キャッシュの使用を無効にします。これにより、メモリ使用量が削減されます。
model.config.pretraining_tp = 1  # 事前訓練時のテンソル並列処理の設定を1にします。これはモデルの並列処理に影響します。
```

</div>
</details>

In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id  # モデルの設定にパディングトークンIDを追加します。
model.config.use_cache = False  # キャッシュの使用を無効にします。これにより、メモリ使用量が削減されます。
model.config.pretraining_tp = 1  # 事前訓練時のテンソル並列処理の設定を1にします。これはモデルの並列処理に影響します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1
```

</div>
<div class="column-right">

# 日本語訳

```python
from datasets import DatasetDict, Dataset  # データセットに関連するクラスをインポートします。

def data_preprocesing(row):  # 各行に対してデータ前処理を行う関数を定義します。
    return tokenizer(row['text'], padding='max_length', truncation=True, max_length=CFG.MAX_LENGTH, return_tensors='np')  # テキストをトークン化し、パディング、切り捨て、トーチテンソルで返します。

# pandasデータフレームからデータセットを作成します。
dataset = Dataset.from_pandas(train)  
# データセットをトークン化し、'text'列を削除します。
tokenized_data = dataset.map(data_preprocesing, batched=True, remove_columns=['text'])  
# トークン化されたデータのフォーマットをtorchに設定します。
tokenized_data.set_format("torch")
```

</div>
</details>

In [ ]:
from datasets import DatasetDict, Dataset  # データセットに関連するクラスをインポートします。

def data_preprocesing(row):  # 各行に対してデータ前処理を行う関数を定義します。
    return tokenizer(row['text'], padding='max_length', truncation=True, max_length=CFG.MAX_LENGTH, return_tensors='np')  # テキストをトークン化し、パディング、切り捨て、トーチテンソルで返します。

# pandasデータフレームからデータセットを作成します。
dataset = Dataset.from_pandas(train)  
# データセットをトークン化し、'text'列を削除します。
tokenized_data = dataset.map(data_preprocesing, batched=True, remove_columns=['text'])  
# トークン化されたデータのフォーマットをtorchに設定します。
tokenized_data.set_format("torch")

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from datasets import DatasetDict, Dataset

def data_preprocesing(row):
    return tokenizer(row['text'], padding='max_length', truncation=True, max_length=CFG.MAX_LENGTH, return_tensors='np')

dataset = Dataset.from_pandas(train)
tokenized_data = dataset.map(data_preprocesing, batched=True, remove_columns=['text'])
tokenized_data.set_format("torch")
```

</div>
<div class="column-right">

# 日本語訳

```python
import gc  # ガーベジコレクションを管理するためのgcライブラリをインポートします。

del train  # メモリ使用量を削減するために、trainデータフレームを削除します。
gc.collect()  # ガーベジコレクションを手動で実行し、不要なメモリを解放します。
```

</div>
</details>

In [ ]:
import gc  # ガーベジコレクションを管理するためのgcライブラリをインポートします。

del train  # メモリ使用量を削減するために、trainデータフレームを削除します。
gc.collect()  # ガーベジコレクションを手動で実行し、不要なメモリを解放します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import gc

del train
gc.collect()
```

</div>
<div class="column-right">

# 日本語訳

```python
from transformers import DataCollatorWithPadding  # パディングを扱うためのデータコレレーターをインポートします。

# トークナイザーを使ってパディングを行うデータコレーション関数を作成します。
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)  # データコレレーターを初期化します。これにより、バッチ内のすべてのサンプルが同じ長さになるようにパディングされます。
```

</div>
</details>

In [ ]:
from transformers import DataCollatorWithPadding  # パディングを扱うためのデータコレレーターをインポートします。

# トークナイザーを使ってパディングを行うデータコレーション関数を作成します。
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)  # データコレレーターを初期化します。これにより、バッチ内のすべてのサンプルが同じ長さになるようにパディングされます。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from transformers import DataCollatorWithPadding

collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)
```

</div>
<div class="column-right">

# 日本語訳

```python
def compute_metrics(evaluations):  # 評価指標を計算する関数を定義します。
    predictions, labels = evaluations  # 評価から予測値とラベルを取得します。
    predictions = np.argmax(predictions, axis=1)  # 予測結果から最大値のインデックスを取得し、クラスラベルに変換します。
    # バランス精度と通常の精度を計算し、辞書形式で返します。
    return {
        'balanced_accuracy': balanced_accuracy_score(predictions, labels),  # バランス精度を計算します。
        'accuracy': accuracy_score(predictions, labels)  # 通常の精度を計算します。
    }
```

</div>
</details>

In [ ]:
def compute_metrics(evaluations):  # 評価指標を計算する関数を定義します。
    predictions, labels = evaluations  # 評価から予測値とラベルを取得します。
    predictions = np.argmax(predictions, axis=1)  # 予測結果から最大値のインデックスを取得し、クラスラベルに変換します。
    # バランス精度と通常の精度を計算し、辞書形式で返します。
    return {
        'balanced_accuracy': balanced_accuracy_score(predictions, labels),  # バランス精度を計算します。
        'accuracy': accuracy_score(predictions, labels)  # 通常の精度を計算します。
    }

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def compute_metrics(evaluations):
    predictions, labels = evaluations
    predictions = np.argmax(predictions, axis=1)
    return {'balanced_accuracy' : balanced_accuracy_score(predictions, labels),
    'accuracy':accuracy_score(predictions,labels)}
```

</div>
<div class="column-right">

# 日本語訳

```python
import torch.nn.functional as F  # PyTorchの関数型APIから機能をインポートします。

class CustomTrainer(Trainer):  # Trainerクラスを拡張したカスタムトレーナークラスを定義します。
    def __init__(self, *args, class_weights=None, **kwargs):  # コンストラクタを定義し、クラス重みを受け取ります。
        super().__init__(*args, **kwargs)  # 親クラスのコンストラクタを呼び出します。
        if class_weights is not None:  # クラス重みが指定されている場合
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)  # テンソルとしてクラス重みを作成し、デバイスに移動します。
        else:
            self.class_weights = None  # クラス重みが指定されていない場合はNoneに設定します。

    def compute_loss(self, model, inputs, return_outputs=False):  # 損失を計算するメソッドを定義します。
#         print(inputs)  # デバッグ用に入力値を表示（コメントアウトされています）。
        labels = inputs.pop("labels").long()  # 入力からラベルを取り出し、long型に変換します。

        outputs = model(**inputs)  # モデルを入力して予測を得ます。

        logits = outputs.get('logits')  # モデルの出力からロジットを取得します。

        if self.class_weights is not None:  # クラス重みが指定されている場合
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)  # 重み付きの交差エントロピー損失を計算します。
        else:
            loss = F.cross_entropy(logits, labels)  # 通常の交差エントロピー損失を計算します。

        return (loss, outputs) if return_outputs else loss  # 出力を返すか、損失のみを返します。
```

</div>
</details>

In [ ]:
import torch.nn.functional as F  # PyTorchの関数型APIから機能をインポートします。

class CustomTrainer(Trainer):  # Trainerクラスを拡張したカスタムトレーナークラスを定義します。
    def __init__(self, *args, class_weights=None, **kwargs):  # コンストラクタを定義し、クラス重みを受け取ります。
        super().__init__(*args, **kwargs)  # 親クラスのコンストラクタを呼び出します。
        if class_weights is not None:  # クラス重みが指定されている場合
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)  # テンソルとしてクラス重みを作成し、デバイスに移動します。
        else:
            self.class_weights = None  # クラス重みが指定されていない場合はNoneに設定します。

    def compute_loss(self, model, inputs, return_outputs=False):  # 損失を計算するメソッドを定義します。
#         print(inputs)  # デバッグ用に入力値を表示（コメントアウトされています）。
        labels = inputs.pop("labels").long()  # 入力からラベルを取り出し、long型に変換します。

        outputs = model(**inputs)  # モデルを入力して予測を得ます。

        logits = outputs.get('logits')  # モデルの出力からロジットを取得します。

        if self.class_weights is not None:  # クラス重みが指定されている場合
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)  # 重み付きの交差エントロピー損失を計算します。
        else:
            loss = F.cross_entropy(logits, labels)  # 通常の交差エントロピー損失を計算します。

        return (loss, outputs) if return_outputs else loss  # 出力を返すか、損失のみを返します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import torch.nn.functional as F

class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):
#         print(inputs)
        labels = inputs.pop("labels").long()

        outputs = model(**inputs)

        logits = outputs.get('logits')

        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss
```

</div>
<div class="column-right">

# 日本語訳

```python
training_args = TrainingArguments(  # トレーニングの引数を設定するためのTrainingArgumentsオブジェクトを作成します。
    output_dir='sentiment_classification',  # モデルの出力先ディレクトリを指定します。
    learning_rate=1e-4,  # 学習率を設定します。
    per_device_train_batch_size=8,  # 各デバイスでのトレーニングバッチサイズを設定します。
    per_device_eval_batch_size=8,  # 各デバイスでの評価バッチサイズを設定します。
    num_train_epochs=1,  # トレーニングのエポック数を設定します。
    logging_steps=1,  # ロギングを行うステップ間隔を設定します。
    weight_decay=0.01,  # 重み減衰率を設定します。これにより過学習を防ぎます。
    evaluation_strategy='epoch',  # 評価の戦略としてエポック毎に評価を設定します。
    save_strategy='epoch',  # モデルの保存戦略としてエポック毎に保存を設定します。
    load_best_model_at_end=True,  # トレーニングの終了時に最良のモデルを読み込む設定です。
    report_to="none"  # ロギングを行わない設定です。
)
```

</div>
</details>

In [ ]:
training_args = TrainingArguments(  # トレーニングの引数を設定するためのTrainingArgumentsオブジェクトを作成します。
    output_dir='sentiment_classification',  # モデルの出力先ディレクトリを指定します。
    learning_rate=1e-4,  # 学習率を設定します。
    per_device_train_batch_size=8,  # 各デバイスでのトレーニングバッチサイズを設定します。
    per_device_eval_batch_size=8,  # 各デバイスでの評価バッチサイズを設定します。
    num_train_epochs=1,  # トレーニングのエポック数を設定します。
    logging_steps=1,  # ロギングを行うステップ間隔を設定します。
    weight_decay=0.01,  # 重み減衰率を設定します。これにより過学習を防ぎます。
    evaluation_strategy='epoch',  # 評価の戦略としてエポック毎に評価を設定します。
    save_strategy='epoch',  # モデルの保存戦略としてエポック毎に保存を設定します。
    load_best_model_at_end=True,  # トレーニングの終了時に最良のモデルを読み込む設定です。
    report_to="none"  # ロギングを行わない設定です。
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
training_args = TrainingArguments(
    output_dir = 'sentiment_classification',
    learning_rate = 1e-4,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 1,
    logging_steps=1,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    report_to="none"
)
```

</div>
<div class="column-right">

# 日本語訳

```python
trainer = CustomTrainer(  # カスタムトレーナーのインスタンスを作成します。
    model=model,  # 使用するモデルを指定します。
    args=training_args,  # トレーニング引数を指定します。
    train_dataset=tokenized_data,  # トレーニングデータセットを指定します。
    tokenizer=tokenizer,  # 使用するトークナイザーを指定します。
    data_collator=collate_fn,  # データコレーターを指定します。バッチのパディングを行います。
    compute_metrics=compute_metrics,  # 評価指標を計算する関数を指定します。
#     class_weights=class_weights,  # ここでクラス重みを使用する場合は、コメントを外します。
)

train_result = trainer.train()  # トレーニングを開始し、結果を取得します。
```

</div>
</details>

In [ ]:
trainer = CustomTrainer(  # カスタムトレーナーのインスタンスを作成します。
    model=model,  # 使用するモデルを指定します。
    args=training_args,  # トレーニング引数を指定します。
    train_dataset=tokenized_data,  # トレーニングデータセットを指定します。
    tokenizer=tokenizer,  # 使用するトークナイザーを指定します。
    data_collator=collate_fn,  # データコレーターを指定します。バッチのパディングを行います。
    compute_metrics=compute_metrics,  # 評価指標を計算する関数を指定します。
#     class_weights=class_weights,  # ここでクラス重みを使用する場合は、コメントを外します。
)

train_result = trainer.train()  # トレーニングを開始し、結果を取得します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data,
    tokenizer = tokenizer,
    data_collator = collate_fn,
    compute_metrics = compute_metrics,
#     class_weights=class_weights,
)

train_result = trainer.train()
```

</div>
<div class="column-right">

# 日本語訳

```python

```

</div>
</details>